In [ ]:
import sys
sys.path.append("..")

from snack import FeatureSpace, Snack, train_model

%load_ext autoreload
%autoreload 2

In [ ]:
dataset_path = "/home/jovyan/nkiselev/studying/bioinformatics/code/data"

# Инициализируем пространство признаков
feature_space = FeatureSpace()

# Инициализируем модель с метрикой
metric = Snack(feature_space=feature_space)

# Запускаем обучение
train_model(
    model=metric,
    dataset_path=dataset_path,
    num_epochs=10,
    learning_rate=0.001,
)


In [ ]:
from Bio.Align import substitution_matrices

# Загружаем матрицы PAM и BLOSUM
pam250 = substitution_matrices.load('PAM250')
blosum62 = substitution_matrices.load('BLOSUM62')

Для того, чтобы сравнить свою метрику с метриками PAM и BLOSUM, вам нужно сначала понять, что эти метрики делают и как они работают, а затем сравнить их с вашей собственной метрикой.

1. PAM и BLOSUM: Обзор
PAM (Point Accepted Mutation): Это матрица, основанная на вероятностях замен аминокислот в ходе эволюции. Чем больше значение в матрице, тем более вероятна замена аминокислот в ходе эволюции.

BLOSUM (Blocks Substitution Matrix): Это еще одна популярная матрица замещений, но она основывается на анализе зафиксированных блоков выравниваний, и замещения в этих блоках, которые встречаются чаще, имеют более высокие баллы.

2. Как сравнить свою метрику с PAM и BLOSUM?
Для сравнения можно использовать несколько подходов:

Прямое сравнение значений между матрицами:
Вы можете рассчитать дистанцию между двумя аминокислотами по своей метрике, а затем сравнить её с результатами для той же пары аминокислот в матрицах PAM и BLOSUM. Это можно сделать для всех аминокислотных пар и посмотреть, насколько ваша метрика сходна с PAM и BLOSUM.

Сравнение с использованием выравниваний:
Можно использовать вашу метрику для выравнивания последовательностей и сравнить полученные результаты с теми, что получаются с использованием матриц PAM и BLOSUM. Это позволит вам оценить, как ваша метрика влияет на выравнивание.

Параметры на основе статистики:
Можно провести статистическое сравнение между матрицами (например, используя корреляцию) для определения того, насколько ваша метрика схожа с PAM или BLOSUM.

In [ ]:
# Сравнение с матрицами PAM и BLOSUM
def compare_matrices(aa1, aa2):
    custom_distance = metric(aa1, aa2)
    pam_score = pam250.get((aa1, aa2), pam250.get((aa2, aa1), 0))
    blosum_score = blosum62.get((aa1, aa2), blosum62.get((aa2, aa1), 0))

    return custom_distance, pam_score, blosum_score

# Пример для аминокислот 'A' и 'G'
aa1, aa2 = 'A', 'G'
custom_distance, pam_score, blosum_score = compare_matrices(aa1, aa2)

print(f"Custom distance: {custom_distance}")
print(f"PAM250 score: {pam_score}")
print(f"BLOSUM62 score: {blosum_score}")


Для того чтобы оценить качество выравнивания с использованием вашей метрики, PAM и BLOSUM, можно использовать следующие подходы:

Вычисление общей стоимости выравнивания для всех пар:

Для каждой пары последовательностей можно вычислить стоимость выравнивания с использованием каждой метрики (вашей, PAM и BLOSUM).

Оценка точности выравнивания:

Для каждой метрики выравнивание будет иметь определенную стоимость. Чтобы провести сравнение, можно вычислить, какая метрика дает наилучшие результаты (например, наименьшую стоимость выравнивания).

Оценка с помощью эталонных данных:

Если у вас есть эталонное выравнивание для пар, вы можете сравнить ваши выравнивания с эталонными по различным метрикам (например, используя согласованность или сходство с эталоном).

In [ ]:
import numpy as np

def needleman_wunsch_custom(seq1, seq2, metric):
    len1, len2 = len(seq1), len(seq2)

    # Инициализация матрицы
    dp = np.zeros((len1 + 1, len2 + 1))

    # Заполнение первой строки и первого столбца
    for i in range(1, len1 + 1):
        dp[i][0] = dp[i - 1][0] + metric(seq1[i - 1], "-")
    for j in range(1, len2 + 1):
        dp[0][j] = dp[0][j - 1] + metric("-", seq2[j - 1])

    # Заполнение остальной матрицы
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match = dp[i - 1][j - 1] + metric(seq1[i - 1], seq2[j - 1])
            delete = dp[i - 1][j] + metric(seq1[i - 1], "-")
            insert = dp[i][j - 1] + metric("-", seq2[j - 1])
            dp[i][j] = min(match, delete, insert)

    # Восстановление выравнивания
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = len1, len2
    while i > 0 and j > 0:
        if dp[i][j] == dp[i - 1][j - 1] + metric(seq1[i - 1], seq2[j - 1]):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif dp[i][j] == dp[i - 1][j] + metric(seq1[i - 1], "-"):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append("-")
            i -= 1
        else:
            aligned_seq1.append("-")
            aligned_seq2.append(seq2[j - 1])
            j -= 1

    while i > 0:
        aligned_seq1.append(seq1[i - 1])
        aligned_seq2.append("-")
        i -= 1

    while j > 0:
        aligned_seq1.append("-")
        aligned_seq2.append(seq2[j - 1])
        j -= 1

    return "".join(reversed(aligned_seq1)), "".join(reversed(aligned_seq2))


In [ ]:
def needleman_wunsch_pam_blosum(seq1, seq2, matrix):
    len1, len2 = len(seq1), len(seq2)

    # Инициализация матрицы
    dp = np.zeros((len1 + 1, len2 + 1))

    # Заполнение первой строки и первого столбца
    for i in range(1, len1 + 1):
        dp[i][0] = dp[i - 1][0] + matrix.get((seq1[i - 1], "-"), matrix.get(("-", seq1[i - 1]), 0))
    for j in range(1, len2 + 1):
        dp[0][j] = dp[0][j - 1] + matrix.get(("-", seq2[j - 1]), matrix.get((seq2[j - 1], "-"), 0))

    # Заполнение остальной матрицы
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            match = dp[i - 1][j - 1] + matrix.get((seq1[i - 1], seq2[j - 1]), matrix.get((seq2[j - 1], seq1[i - 1]), 0))
            delete = dp[i - 1][j] + matrix.get((seq1[i - 1], "-"), matrix.get(("-", seq1[i - 1]), 0))
            insert = dp[i][j - 1] + matrix.get(("-", seq2[j - 1]), matrix.get((seq2[j - 1], "-"), 0))
            dp[i][j] = min(match, delete, insert)

    # Восстановление выравнивания
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = len1, len2
    while i > 0 and j > 0:
        if dp[i][j] == dp[i - 1][j - 1] + matrix.get((seq1[i - 1], seq2[j - 1]), matrix.get((seq2[j - 1], seq1[i - 1]), 0)):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif dp[i][j] == dp[i - 1][j] + matrix.get((seq1[i - 1], "-"), matrix.get(("-", seq1[i - 1]), 0)):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append("-")
            i -= 1
        else:
            aligned_seq1.append("-")
            aligned_seq2.append(seq2[j - 1])
            j -= 1

    while i > 0:
        aligned_seq1.append(seq1[i - 1])
        aligned_seq2.append("-")
        i -= 1

    while j > 0:
        aligned_seq1.append("-")
        aligned_seq2.append(seq2[j - 1])
        j -= 1

    return "".join(reversed(aligned_seq1)), "".join(reversed(aligned_seq2))


In [ ]:
# Сравнение выравниваний для одной пары последовательностей
seq1, seq2 = 'AGCT', 'AGT'

# Выравнивание с вашей метрикой
aligned_custom1, aligned_custom2 = needleman_wunsch_custom(seq1, seq2, metric)

# Выравнивание с PAM
aligned_pam1, aligned_pam2 = needleman_wunsch_pam_blosum(seq1, seq2, pam250)

# Выравнивание с BLOSUM
aligned_blosum1, aligned_blosum2 = needleman_wunsch_pam_blosum(seq1, seq2, blosum62)

# Вывод результатов
print("Aligned with custom metric:")
print(aligned_custom1)
print(aligned_custom2)

print("\nAligned with PAM:")
print(aligned_pam1)
print(aligned_pam2)

print("\nAligned with BLOSUM:")
print(aligned_blosum1)
print(aligned_blosum2)


Для расширения сравнения ваших метрик с другими, менее специфичными метриками, которые не учитывают биологическую информацию, можно добавить несколько классических метрик для строк, таких как:

Метрика Левенштейна (расстояние редактирования) — измеряет минимальное количество операций (вставка, удаление, замена), необходимых для преобразования одной строки в другую.

Метрика Хэмминга — измеряет количество позиций, в которых символы двух строк различаются. Эта метрика работает только для строк одинаковой длины.

Джаккардова мера сходства — оценивает схожесть двух строк как отношение размера их пересечения к размеру объединения.

Косинусное сходство — используется для оценки схожести между двумя строками, представленных как векторные представления (например, через частотные распределения символов).

Реализация этих метрик и добавление их в процесс сравнения
1. Метрика Левенштейна (расстояние редактирования)
Метрика Левенштейна измеряет минимальное количество операций, которые необходимо выполнить для преобразования одной строки в другую.

In [ ]:
import numpy as np

def levenshtein_distance(seq1, seq2):
    """Вычисление расстояния Левенштейна (расстояния редактирования)"""
    len1, len2 = len(seq1), len(seq2)
    dp = np.zeros((len1 + 1, len2 + 1))
    
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j
    
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = 0 if seq1[i - 1] == seq2[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,  # Удаление
                dp[i][j - 1] + 1,  # Вставка
                dp[i - 1][j - 1] + cost  # Замена
            )
    
    return dp[len1][len2]


2. Метрика Хэмминга
Метрика Хэмминга измеряет количество позиций, в которых символы двух строк различаются (предполагается, что строки имеют одинаковую длину).

In [ ]:
def hamming_distance(seq1, seq2):
    """Вычисление расстояния Хэмминга"""
    if len(seq1) != len(seq2):
        raise ValueError("Строки должны быть одинаковой длины для вычисления расстояния Хэмминга.")
    
    return sum(el1 != el2 for el1, el2 in zip(seq1, seq2))


3. Джаккардова мера сходства
Джаккардова мера оценивает схожесть двух строк как отношение их пересечения к их объединению. Для строк можно преобразовать строки в множества символов.

In [ ]:
def jaccard_similarity(seq1, seq2):
    """Вычисление Джаккардовой меры сходства"""
    set1, set2 = set(seq1), set(seq2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    
    return intersection / union if union != 0 else 0


4. Косинусное сходство
Для вычисления косинусного сходства между строками можно преобразовать их в векторы с использованием частот символов (или биграмм).

In [ ]:
from collections import Counter
from math import sqrt

def cosine_similarity(seq1, seq2):
    """Вычисление косинусного сходства"""
    def vectorize(seq):
        return Counter(seq)
    
    vec1 = vectorize(seq1)
    vec2 = vectorize(seq2)
    
    intersection = set(vec1) & set(vec2)
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    
    sum1 = sum([vec1[x]**2 for x in vec1])
    sum2 = sum([vec2[x]**2 for x in vec2])
    denominator = sqrt(sum1) * sqrt(sum2)
    
    return numerator / denominator if denominator != 0 else 0


5. Интеграция всех метрик в процесс сравнения выравниваний
Теперь, когда у нас есть все эти метрики, мы можем интегрировать их в общий процесс сравнения для всех метрик (вашей, PAM, BLOSUM и новых метрик).

In [ ]:
def compare_alignments(seq1, seq2, model, pam_matrix, blosum_matrix):
    """Сравнение выравниваний с использованием различных метрик"""
    
    # Выравнивание с вашей метрикой
    aligned_custom1, aligned_custom2 = needleman_wunsch_custom(seq1, seq2, model)

    # Выравнивание с PAM
    aligned_pam1, aligned_pam2 = needleman_wunsch_pam_blosum(seq1, seq2, pam_matrix)

    # Выравнивание с BLOSUM
    aligned_blosum1, aligned_blosum2 = needleman_wunsch_pam_blosum(seq1, seq2, blosum_matrix)

    # Вычисление расстояния для каждой метрики
    lev_custom = levenshtein_distance(aligned_custom1, aligned_custom2)
    lev_pam = levenshtein_distance(aligned_pam1, aligned_pam2)
    lev_blosum = levenshtein_distance(aligned_blosum1, aligned_blosum2)

    ham_custom = hamming_distance(aligned_custom1, aligned_custom2)
    ham_pam = hamming_distance(aligned_pam1, aligned_pam2)
    ham_blosum = hamming_distance(aligned_blosum1, aligned_blosum2)

    jac_custom = jaccard_similarity(aligned_custom1, aligned_custom2)
    jac_pam = jaccard_similarity(aligned_pam1, aligned_pam2)
    jac_blosum = jaccard_similarity(aligned_blosum1, aligned_blosum2)

    cos_custom = cosine_similarity(aligned_custom1, aligned_custom2)
    cos_pam = cosine_similarity(aligned_pam1, aligned_pam2)
    cos_blosum = cosine_similarity(aligned_blosum1, aligned_blosum2)

    # Вывод результатов
    print("Levenshtein distance:")
    print(f"Custom: {lev_custom}, PAM: {lev_pam}, BLOSUM: {lev_blosum}")
    
    print("\nHamming distance:")
    print(f"Custom: {ham_custom}, PAM: {ham_pam}, BLOSUM: {ham_blosum}")
    
    print("\nJaccard similarity:")
    print(f"Custom: {jac_custom}, PAM: {jac_pam}, BLOSUM: {jac_blosum}")
    
    print("\nCosine similarity:")
    print(f"Custom: {cos_custom}, PAM: {cos_pam}, BLOSUM: {cos_blosum}")


In [ ]:
# Предположим, что у вас есть последовательности seq1 и seq2
seq1, seq2 = 'AGCT', 'AGT'

# Сравниваем выравнивания
compare_alignments(seq1, seq2, metric, pam250, blosum62)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Для визуализации матриц, преобразуем метрику в форму матрицы для heatmap
def create_matrix_from_metric(metric, amino_acids):
    size = len(amino_acids)
    matrix = np.zeros((size, size))
    for i, aa1 in enumerate(amino_acids):
        for j, aa2 in enumerate(amino_acids):
            matrix[i, j] = metric(aa1, aa2)
    return matrix

# Определим список аминокислот для вычисления метрики
amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # 20 стандартных аминокислот

# Получаем матрицы для трех метрик
custom_matrix = create_matrix_from_metric(metric, amino_acids)
pam_matrix_values = np.array([[pam250.get((aa1, aa2), pam250.get((aa2, aa1), 0)) for aa2 in amino_acids] for aa1 in amino_acids])
blosum_matrix_values = np.array([[blosum62.get((aa1, aa2), blosum62.get((aa2, aa1), 0)) for aa2 in amino_acids] for aa1 in amino_acids])

# Создаем фигуру для отображения
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Визуализируем матрицы
sns.heatmap(custom_matrix, ax=axes[0], cmap="coolwarm", annot=False, xticklabels=amino_acids, yticklabels=amino_acids)
axes[0].set_title("Custom Metric")

sns.heatmap(pam_matrix_values, ax=axes[1], cmap="coolwarm", annot=False, xticklabels=amino_acids, yticklabels=amino_acids)
axes[1].set_title("PAM250")

sns.heatmap(blosum_matrix_values, ax=axes[2], cmap="coolwarm", annot=False, xticklabels=amino_acids, yticklabels=amino_acids)
axes[2].set_title("BLOSUM62")

# Настроим отображение
plt.tight_layout()
plt.show()